In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Đọc danh sách URL từ tệp
url_list = []
with open('http.csv', 'r') as file:
    url_list = [line.strip() for line in file]

# Tiêu đề User-Agent giả lập một trình duyệt
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Khởi tạo danh sách để lưu thông tin sản phẩm
all_product_list = []

# Duyệt qua từng URL và lấy thông tin
for url in url_list:
    # Lấy tên hãng từ URL
    sub_category = url.split('-')[-1].capitalize()
    
    # Gửi yêu cầu đến trang web với tiêu đề User-Agent và thời gian chờ (timeout)
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()  # Kiểm tra lỗi

    # Phân tích HTML của trang web
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm các thẻ chứa thông tin sản phẩm
    products = soup.find_all('li', class_='item')

    # Duyệt qua từng sản phẩm và lấy thông tin
    for product in products:
        try:
            # Kiểm tra và tìm thẻ chứa tên sản phẩm
            title_tag = product.find('h3')
            title = title_tag.text.strip() if title_tag else 'No title available'

            # Kiểm tra và tìm thẻ chứa giá sản phẩm
            price_tag = product.find('strong')
            price = price_tag.text.strip().replace('₫', '').strip() if price_tag else 'No price available'
            
            # Kiểm tra và tìm thẻ chứa URL ảnh
            img_div = product.find('div', class_='item-img item-img_42')
            img_tag = img_div.find('img') if img_div else None
            image_url = img_tag['src'] if img_tag and 'src' in img_tag.attrs and '600x600' in img_tag['src'] else 'No image URL'

            # Lấy thông tin chi tiết sản phẩm từ phần mô tả ngắn
            description_tag = product.find('div', class_='utility')
            if description_tag:
                description_items = description_tag.find_all('p')
                description = ' | '.join(item.text.strip() for item in description_items) if description_items else 'No description available'
            else:
                description = 'No description available'
            
            # Lưu thông tin sản phẩm vào danh sách nếu không có lỗi
            if title != 'No title available' and price != 'No price available' and image_url != 'No image URL' and description != 'No description available':
                all_product_list.append({
                    'category': 'Điện thoại',
                    'sub_category': sub_category,
                    'title': title,
                    'description': description,
                    'price': price,
                    'image_url': image_url
                })
        except Exception as e:
            print(f"Error processing product: {e}")
            continue

# Tạo DataFrame từ danh sách sản phẩm
df = pd.DataFrame(all_product_list)

# Lưu DataFrame vào file CSV
output_path = 'products.csv'
df.to_csv(output_path, index=False)

print(df.head())


     category sub_category              title  \
0  Điện thoại       Iphone  iPhone 15 Pro Max   
1  Điện thoại       Iphone          iPhone 13   
2  Điện thoại       Iphone      iPhone 15 Pro   
3  Điện thoại       Iphone     iPhone 15 Plus   
4  Điện thoại       Iphone          iPhone 15   

                                         description       price  \
0  Chip Apple A17 Pro 6 nhân | RAM: 8 GB | Dung l...  29.590.000   
1  Chip Apple A15 Bionic | RAM: 4 GB | Dung lượng...  13.690.000   
2  Chip Apple A17 Pro 6 nhân | RAM: 8 GB | Dung l...  24.690.000   
3  Chip Apple A16 Bionic | RAM: 6 GB | Dung lượng...  22.190.000   
4  Chip Apple A16 Bionic | RAM: 6 GB | Dung lượng...  19.090.000   

                                           image_url  
0  https://cdn.tgdd.vn/Products/Images/42/305658/...  
1  https://cdn.tgdd.vn/Products/Images/42/223602/...  
2  https://cdn.tgdd.vn/Products/Images/42/299033/...  
3  https://cdn.tgdd.vn/Products/Images/42/303891/...  
4  https://cdn.tgdd.v